How to change file structure of large folder that is backed up with Rsync
-------------------------------------------------------------------------

I found a nice blog on how to change the file structure of a large folder that is backed up with Rsync. The original post can be found [here](https://lincolnloop.com/blog/detecting-file-moves-renames-rsync/). I am making these notes incase it ever dissappears. 

   1. Start by making sure you have the same data at both ends by running the normal backup script, e.g. 
   ```
   %> rsync -avzHP ~/local/backup_folder remote_host:remote/backup_folder
   ```
   1. Make a copy of the local folder as below:
   ```
   %> cd ~/local
   %> cp -rlp backup_folder backup_folder_new
   ```
   The copy should be ver fast as it is using ```-r``` (recursive), ```-l``` (link, instead of copy) and ```-p``` (preserve mode, ownership, timestamps etc).
   1. Reorganise the ```backup_folder_new``` as you please, but don't edit the original ```backup_folder```
   1. Now run Rsync as follows:
   ```
   %> rsync -avzHPn --stats --no-inc-recursive ~/local/backup_folder ~/local/backup_folder_new remote_host:remote/backup_folder
   ```
   Notice the ```-n``` and ```--stats``` options. The ```-n``` does a dry run and ```--stats``` will show the amount of data to be transfered. This should be 0 Bytes if files were only moved and nothing was added or deleted. By transferring both trees at once and by turning off incremental recursion, rsync collects all hard-links before it transfers anything. It is now able to reconstruct backup_folder_new on the remote machine in fraction of the time to transfer the data.
   1. Finally, once you are happy, remove the ```-n``` flag in the command above to let Rsync make the changes. 
   1. Then to tidy up do the following at both ends:
   ```
   %> mv backup_folder backup_folder_old
   %> mv backup_folder_new backup_folder
   ```
   1. The ```backup_folder_old``` can be kept for as long as you want, or simply delete it after the transfer. 